[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/CCS-ZCU/pribehy-dat/blob/master/scripts/ocr.ipynb)

Tento soubor je součástí sestavy elektronických studijních opor [Příběhy dat: Výpočetní přístupy ke studiu kultury a společnosti](https://github.com/CCS-ZCU/pribehy-dat/tree/master). 

# OCR: Rozpoznávání znaků textu

**autor**: *Vojtěch Kaše* (kase@ff.zcu.cz)

[![](https://ccs.zcu.cz/wp-content/uploads/2021/10/cropped-ccs-logo_black_space_240x240.png)](https://ccs.zcu.cz)

## Úvod a cíle kapitoly

Tato kapitola ukazuje základy technologie optického rozpoznávání znaků (tzv. OCR=Optical Charater Recognition). K OCR lze v Pythonu použít knihovnu `pytesseract`, postavené na nástroji Tesseract. Nejprve je tudíž potřeba mít nainstallovaný Tesseract, což není triviální úloha, neboť závisí na konkrétní verzi operečního systému. Instrukce pro windows a linux je možné nalézt zde: https://github.com/UB-Mannheim/tesseract/wiki. Pro mac nám napoví např. ChatGPT. Jelikož pracuji na macu, zde nabízím příkazy k instalaci Tesseractu pomocí terminálového instalačního nástroje `brew`: 
``` 
brew install tessaract
brew install tessaract-lang
```

Pro ukázku si načteme jeden dokument z projektu scriptum.cz, který neprošel OCR analýzou:

## Cvičení

In [ ]:
#!pip install PyMuPDF
import fitz
import requests
import io
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import pytesseract
import cv2
import matplotlib.patches as patches
import re
from bs4 import BeautifulSoup
import pandas as pd
import nltk

In [ ]:
url = "https://scriptum.cz/soubory/scriptum/svedectvi/svedectvi_1966_29.pdf" # "https://scriptum.cz/soubory/scriptum/komunikace/prazske_komunikace_1987_01.pdf"

pdf_object = io.BytesIO(requests.get(url).content)
doc = fitz.open("pdf", pdf_object.read())
doc.page_count

Pro potřeby testování opět vybereme jednu náhodnou stránku a vytvoříme z ní objekt obrázku.

In [ ]:
p = doc.load_page(20)
pix = p.get_pixmap()
np_array = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.height, pix.width, pix.n)
fig, ax = plt.subplots(dpi=300)
ax.imshow(np_array)
ax.set_axis_off()

In [ ]:
%%time
pytesseract.image_to_string(np_array, lang="ces")

Tuto proceduru nyní snadno aplikujeme na všechny stránky v daném dokumentu. Tato procedura však již může zabrat nezanedbatelný čas. 

In [ ]:
%%time
text = ""
for p in doc:
    pix = p.get_pixmap()
    np_array = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.height, pix.width, pix.n)
    str = pytesseract.image_to_string(np_array, lang="ces") + "[pagebreak]"
    text += str 

Podívejme se nyní na výsledný text. Z kolika sestává znaků?

In [ ]:
len(text)

In [ ]:
text[16000:17000]

Vidíme, že výsledky nejsou kdovíjaké.

### Morfologické transformace

Často se potýkáme s tím, že podkladové obrázky (skeny jednotlivých stran) nejsou v dobré kvalitě. Existuje však řada algoritmů "out-of-the-box", které mohou kvalitu obrázků změnit, např. zvýšením kontrastu.

In [ ]:
p = doc.load_page(20)
pix = p.get_pixmap()
np_array = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.height, pix.width, pix.n)

In [ ]:
kernel = np.ones((2, 2), np.uint8)
np_array_corrected = cv2.erode(np_array, kernel, iterations=1)
fig, ax = plt.subplots(dpi=300)
ax.imshow(np_array_corrected)
ax.set_axis_off()
print(pytesseract.image_to_string(np_array_corrected, lang="ces"))

In [ ]:
# DILATION
kernel = np.ones((1, 1), np.uint8)
np_array_corrected = cv2.dilate(np_array, kernel, iterations=1)
fig, ax = plt.subplots(dpi=300)
ax.imshow(np_array_corrected)
ax.set_axis_off()
print(pytesseract.image_to_string(np_array_corrected, lang="ces"))

In [ ]:

# EROSION
kernel = np.ones((1, 1), np.uint8)
np_array_corrected = cv2.erode(np_array, kernel, iterations=1)
fig, ax = plt.subplots(dpi=300)
ax.imshow(np_array_corrected)
ax.set_axis_off()
print(pytesseract.image_to_string(np_array_corrected, lang="ces"))

In [ ]:

# CLOSING
kernel = np.ones((1, 1), np.uint8)
np_array_corrected = cv2.morphologyEx(np_array, cv2.MORPH_CLOSE, kernel)
fig, ax = plt.subplots(dpi=300)
ax.imshow(np_array_corrected)
ax.set_axis_off()
print(pytesseract.image_to_string(np_array_corrected, lang="ces"))

In [ ]:
kernel = np.ones((1,1),np.uint8)
opening = cv2.morphologyEx(np_array, cv2.MORPH_OPEN, kernel)
np_array_corrected = cv2.morphologyEx(np_array, cv2.MORPH_CLOSE, kernel)
fig, ax = plt.subplots(dpi=300)
ax.imshow(np_array_corrected)
ax.set_axis_off()
print(pytesseract.image_to_string(np_array_corrected, lang="ces"))